<a href="https://colab.research.google.com/github/MarkovMarkowitz/MarkovMarkowitz/blob/main/ON4_RapidAPI_YFinanceJSON_Read_Hourly_Data_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import requests

url = "https://yahoo-finance127.p.rapidapi.com/historic/TUPRS.IS/1h/3mo"

headers = {
	"X-RapidAPI-Key": "c4100a6817msh6100fdd804b1ae8p1defa4jsn72757c98b3c0",
	"X-RapidAPI-Host": "yahoo-finance127.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())

{'meta': {'currency': 'TRY', 'symbol': 'TUPRS.IS', 'exchangeName': 'IST', 'instrumentType': 'EQUITY', 'firstTradeDate': 957940200, 'regularMarketTime': 1692371393, 'gmtoffset': 10800, 'timezone': 'TRT', 'exchangeTimezoneName': 'Europe/Istanbul', 'regularMarketPrice': 127, 'chartPreviousClose': 65.55, 'previousClose': 123, 'scale': 3, 'priceHint': 2, 'currentTradingPeriod': {'pre': {'timezone': 'TRT', 'end': 1692340200, 'start': 1692340200, 'gmtoffset': 10800}, 'regular': {'timezone': 'TRT', 'end': 1692370800, 'start': 1692340200, 'gmtoffset': 10800}, 'post': {'timezone': 'TRT', 'end': 1692370800, 'start': 1692370800, 'gmtoffset': 10800}}, 'tradingPeriods': [[{'timezone': 'TRT', 'end': 1684767600, 'start': 1684737000, 'gmtoffset': 10800}], [{'timezone': 'TRT', 'end': 1684854000, 'start': 1684823400, 'gmtoffset': 10800}], [{'timezone': 'TRT', 'end': 1684940400, 'start': 1684909800, 'gmtoffset': 10800}], [{'timezone': 'TRT', 'end': 1685026800, 'start': 1684996200, 'gmtoffset': 10800}], [{

In [65]:
if response.status_code == 200:
    data = response.json()

    # Extract the "close" data
    close_data = data['indicators']['quote'][0]['close']

    # # Create a DataFrame from the "close" data
    # df = pd.DataFrame({'Close': close_data})

    # print(df)

In [71]:
timestamps = data["timestamp"]


# Convert timestamps to datetime
timestamps = [pd.to_datetime(ts, unit="s") for ts in timestamps]
timestamps = [ts + pd.Timedelta(hours=3) for ts in timestamps]

# Create a DataFrame
#df = pd.DataFrame({"Date-Time": timestamps, "Close": close_data})
df = pd.DataFrame({"Close": close_data}, index=timestamps)

df[df.index >"2023-08-18 00:00:00"]

,Close
2023-08-18 09:30:00,131.899994
2023-08-18 10:30:00,131.000000
2023-08-18 11:30:00,129.699997
2023-08-18 12:30:00,130.399994
2023-08-18 13:30:00,129.800003
2023-08-18 14:30:00,129.000000
2023-08-18 15:30:00,128.199997
2023-08-18 16:30:00,126.900002
2023-08-18 17:30:00,127.900002
2023-08-18 18:00:00,127.000000


In [74]:
# Create an empty list to store the split DataFrames
split_dataframes = []

# Define the start and end times
start_time = pd.to_datetime("09:30:00").time()
end_time = pd.to_datetime("18:00:00").time()

# Iterate through the DataFrame and split it
for date in df.index.date.unique():
    daily_data = df[df.index.date == date]  # Filter data for the current date

    # Filter data for the specified time range
    daily_data = daily_data[(daily_data.index.time >= start_time) & (daily_data.index.time <= end_time)]

    if not daily_data.empty:
        split_dataframes.append(daily_data)

# Print the split DataFrames
for i, split_df in enumerate(split_dataframes):
    print(f"DataFrame {i + 1}:\n{split_df}\n")


AttributeError: ignored

In [67]:
# ///////////////////////////////////////////////////////////
# FUNCTIONS FOR ALPHA VANTAGE
# ///////////////////////////////////////////////////////////
TIME_SERIES_INTRADAY = 'TIME_SERIES_INTRADAY'
TIME_SERIES_INTRADAY_EXTENDED = 'TIME_SERIES_INTRADAY_EXTENDED'
TIME_SERIES_DAILY = 'TIME_SERIES_DAILY'
TIME_SERIES_DAILY_ADJUSTED = 'TIME_SERIES_DAILY_ADJUSTED'
TIME_SERIES_WEEKLY = 'TIME_SERIES_WEEKLY'
TIME_SERIES_WEEKLY_ADJUSTED = 'TIME_SERIES_WEEKLY_ADJUSTED'
TIME_SERIES_MONTHLY = 'TIME_SERIES_MONTHLY'
TIME_SERIES_MONTHLY_ADJUSTED = 'TIME_SERIES_MONTHLY_ADJUSTED'

# ///////////////////////////////////////////////////////////
# TIME SERIES FOR STOCK DATA
# ///////////////////////////////////////////////////////////
ONEMIN = 'Time Series (1min)'
FIVEMIN = 'Time Series (5min)'
FIVETEENMIN = 'Time Series (15min)'
THIRTYMIN = 'Time Series (30min)'
HOUR = 'Time Series (60min)'
FIVEMIN = 'Time Series (5min)'
DAILY = 'Time Series (Daily)'
WEEKLY = 'Weekly Time Series'
WEEKLY_ADJ = 'Weekly Adjusted Time Series'
MONTLY = 'Monthly Time Series'
MONTLY_ADJ = 'Monthly Adjusted Time Series'

# /////////////////////////////////////////////////////////
# query timeseries stock data of selected type ////////////
# /////////////////////////////////////////////////////////
def query_timeseries_data(func, symbol, outputsize='full', datatype='json', apikey='AN5SOF59K41U0YQ3'):
    data = {
        "function": func, # Here you can put any of the 'funcs' above
        "symbol": symbol,
        "outputsize": outputsize, # (full) and (compact) are accepted
        "datatype": datatype,
        "apikey": apikey
    }
    return requests.get("https://www.alphavantage.co/query", data).json()

# /////////////////////////////////////////////////////////
# convert timeseries stock data of selected type to json //
# /////////////////////////////////////////////////////////
def get_timeseries_dataframe(json, type):
    df = pd.DataFrame.from_dict(json[type], orient= 'index')
    df.index =  pd.to_datetime(df.index, format='%Y-%m-%d')

    df = df.rename(columns={ '1. open': 'Open', '2. high': 'High', '3. low': 'Low', '4. close': 'Close', '5. volume': 'Volume'})
    df = df.astype({'Open': 'float64', 'High': 'float64', 'Low': 'float64','Close': 'float64','Volume': 'float64',})
    df = df[[ 'Open', 'High', 'Low', 'Close', 'Volume']]
    return df

# Now all you have to do is actually call the funcs
response_json = query_timeseries_data(TIME_SERIES_DAILY, 'MSFT')
stockprice_dataframe = get_timeseries_dataframe(response_json, DAILY)


,Open,High,Low,Close,Volume
2023-08-18,314.49,318.38,311.5508,316.48,24755012.0
2023-08-17,320.54,321.87,316.2100,316.88,21257161.0
2023-08-16,320.80,324.42,319.8035,320.40,20698864.0
2023-08-15,323.00,325.09,320.9034,321.86,16966285.0
2023-08-14,321.39,324.06,320.0800,324.04,18836139.0
...,...,...,...,...,...
1999-11-05,91.81,92.87,90.5000,91.56,35083700.0
1999-11-04,92.31,92.75,90.3100,91.75,27119700.0
1999-11-03,92.94,93.50,91.5000,92.00,22258500.0
1999-11-02,92.75,94.50,91.9400,92.56,23174500.0
